In [5]:
import sqlite3
from collections import Counter
import re
import matplotlib.pyplot as plt


In [4]:
def run_query(database_path, query):
    # Connect to the SQLite database
    # If the database does not exist, it will be created
    connection = sqlite3.connect(database_path)
    
    try:
        # Create a cursor object using the connection
        cursor = connection.cursor()
        
        # Execute the SQL query
        cursor.execute(query)
        
        # Fetch all results from the executed query
        results = cursor.fetchall()
        
        # Print each row in the results
        for row in results:
            print(row)
            
    except sqlite3.Error as e:
        print(f"An error occurred: {e}")
        
    finally:
        
        # Close the connection to the database
        if connection:
            connection.close()


('olly UG (haftungsbeschränkt)',)
('BLUECHILLED Verwaltungs GmbH',)
('Mittelständische Beteiligungsgesellschaft Bremen mbH',)
('Albert Barufe GmbH',)
('ITERGO Informationstechnologie GmbH',)
('Rheinbahn AG',)
('Verwaltung IFÖ Zweite Immobilienfonds für Österreich GmbH',)
('AWS Personalmarketing GmbH',)
('Rembert Rechtsanwälte Rembert Müller Putzier + Partner eingetragene Partnerschaft mit beschränkter Berufshaftung',)
('POX Media Verwaltungs GmbH',)
('HB-DIGITAL GmbH',)
('Neunte Grundstücksverwaltung AHG-Beteiligungs- und Handelsgesellschaft mbH & Co. KG',)
('wpd Windparks Westfalen IV GmbH',)
('Palletways Deutschland GmbH',)
('S-KON Sales Kontor Hamburg GmbH',)
('ElectronicPartner Handel SE',)
('Stadtküche GmbH',)
('SIEBTE PAXAS Treuhand- und Beteiligungsgesellschaft mbH',)
('"ad fontes" Solartechnik GmbH',)
('Ferox Zweite Holding GmbH',)
('Yücebudak IT & Vertrieb UG (haftungsbeschränkt)',)
('Verwaltung NORDCAPITAL Schiffsportfolio 4 GmbH',)
('HCG Hanse Convenience GmbH',)
('FAP Betei

In [ ]:
def analyze_term_frequency(names):
    # Join all names into a single string
    all_names = ' '.join(names)
    
    # Tokenize the string into words using regex to handle different delimiters
    words = re.findall(r'\b\w+\b', all_names.lower())
    
    # Count the frequency of each word using Counter
    term_frequency = Counter(words)
    
    return term_frequency

In [ ]:
# Example usage
database_path = 'openregister.db'  # Path to your SQLite database file
query = 'SELECT name FROM company LIMIT 100;'  # Replace 'your_table' with your actual table name

list_all_names = run_query(database_path, query)

In [ ]:
# Analyze term frequency in names
term_frequency = analyze_term_frequency(list_all_names)

# Print term frequency results
for term, freq in term_frequency.items():
    print(f"'{term}': {freq}")

In [ ]:

def plot_top_terms(term_frequency, top_n=10):
    # Get the most common terms and their frequencies
    common_terms = term_frequency.most_common(top_n)
    
    # Separate terms and frequencies for plotting
    terms, frequencies = zip(*common_terms)
    
    # Create a bar chart
    plt.figure(figsize=(10, 6))
    plt.bar(terms, frequencies, color='skyblue')
    
    # Add title and labels
    plt.title('Top Occurring Terms')
    plt.xlabel('Terms')
    plt.ylabel('Frequency')
    
    # Rotate x-axis labels for better readability
    plt.xticks(rotation=45)
    
    # Show plot
    plt.tight_layout()
    plt.show()
